### There is more confusion on whether grid should be used in machine learning or are we using averages from dataset to setup a model
### Untill that is clear, the ML works stops here

In [2]:
#read all the data and filter for only some
from ml_utils import *
import time

In [3]:
#get the required clusters 
client = get_cluster("UCSB0021", cores = 40)
client.cluster;

/glade/u/home/sbhattarai/.conda/envs/meds-py/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35797 instead
  warnings.warn(


In [4]:
#modify the function if you want to pass the parameter
def read_all_simulation():
    '''prepare cluster list and read to create ensemble(group of data)
    use preprocess to select only certain dimension and a variable'''
    # Start timing
    start_time = time.time()

    # read all simulations as a list
    cluster_list = sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2005-02-01-00000.nc'))
    cluster_list = cluster_list[1:len(cluster_list)]

    # only select latitude, longitude, time, and using this in preprocess steps
    def preprocess(ds):
        '''using this function in xr.open_mfdataset as preprocess
        ensures that when only these four things are selected
        before the data is combined'''
        ds = fix_time(ds)
        ds = ds[['lat', 'lon', 'time', 'LEAFCN', 'LEAFN', 'TSA']]
        return ds
    
    # read the list and load it for the notebook
    ds = xr.open_mfdataset(cluster_list,
                              combine='nested',
                              preprocess=lambda ds: preprocess(ds),
                              parallel=True,
                              concat_dim="ens")

    # End timing
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Time taken to run read_all_simulation(): {:.2f} seconds".format(elapsed_time))
    return ds
   

In [5]:
ds = read_all_simulation()

Time taken to run read_all_simulation(): 21.57 seconds


**The weight_landarea_gridcells function available inside the utils is designed only for dataarray, 
I am changing to work with dataset.**

In [6]:
#read the landarea data
landarea = xr.open_dataset('/glade/campaign/cgd/tss/projects/PPE/helpers/sparsegrid_landarea.nc')['landarea']

def weight_landarea_gridcells(ds, landarea):
    for varname in ds.variables:
        # Skip coordinates and auxiliary variables
        if varname not in ds.coords:
            var = ds[varname]
            # Calculate the weighted mean of the variable over grid cells
            weighted_mean = var.weighted(landarea).mean(dim='gridcell')
            # Assign the weighted mean back to the dataset
            ds[varname] = weighted_mean
    return ds
   

**The dataset is weighted by gridcells at this point** 

In [7]:
ds = weight_landarea_gridcells(ds, landarea)

**Weight the dataset based on yearly average. Note: these functions are designed to workwith dataset and not with datarray and these functions are also different from what is available in utils. We have to change in the utils later**

In [8]:
def yearly_weighted_average(ds):
    for varname in ds.variables:
        # Skip coordinates and auxiliary variables
        if varname not in ds.coords:
            var = ds[varname]
            # Get the array of number of days from the main dataset
            days_in_month = ds['time.daysinmonth']

            # # Multiply each month's data by corresponding days in month
            weighted_month = (days_in_month * var).groupby("time.year").sum(dim='time')

            # # Total days in the year
            total_days = days_in_month.groupby("time.year").sum(dim='time')

            # # Calculate weighted average for the year
            ds[varname] = weighted_month / total_days

    return ds



In [9]:
ds_final  = yearly_weighted_average(ds)
ds_final

<xarray.Dataset>
Dimensions:  (lat: 96, lon: 144, time: 60, year: 5, ens: 500)
Coordinates:
  * lat      (lat) float32 -90.0 -88.11 -86.21 -84.32 ... 84.32 86.21 88.11 90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) object 2005-01-01 00:00:00 ... 2009-12-01 00:00:00
  * year     (year) int64 2005 2006 2007 2008 2009
Dimensions without coordinates: ens
Data variables:
    LEAFCN   (year, ens) float64 dask.array<chunksize=(1, 1), meta=np.ndarray>
    LEAFN    (year, ens) float64 dask.array<chunksize=(1, 1), meta=np.ndarray>
    TSA      (year, ens) float64 dask.array<chunksize=(1, 1), meta=np.ndarray>
Attributes: (12/40)
    title:                                     CLM History file information
    comment:                                   NOTE: None of the variables ar...
    Conventions:                               CF-1.0
    history:                                   created on 08/10/23 00:09:45
    source:                                    Community Terrestrial Systems ...
    hostname:                                  cheyenne
    ...                                        ...
    ctype_urban_pervious_road:                 75
    cft_c3_crop:                               1
    cft_c3_irrigated:                          2
    time_period_freq:                          month_1
    Time_constant_3Dvars_filename:             ./PPEn11_transient_LHC0001.clm...
    Time_constant_3Dvars:                      ZSOI:DZSOI:WATSAT:SUCSAT:BSW:H...

**Make sure not to run single chunk for above functions, either run all of them or none**

##### The whole function of reading all simulations will be used only when all things are ready, for now it is of no use, so can be saved as raw.

In [ ]:
#-----------steps involves
#work with one piece of data only
#split up the chunk
#setup univariate gaussian 
#fit the gaussian cost function between leafcn and
#plot the raw lines without any hyperparamter change

#### The function is tested with single simulation of 1000 dataset only 

In [10]:
#create an array for both leafn and Leafcn
leafn =  ds_final["LEAFN"].values
leafcn = ds_final["LEAFCN"].values

lat = ds_final['lat'].values
lon = ds_final ['lon'].values
time = ds_final ['time'].values


In [27]:
time

array([cftime.DatetimeNoLeap(2005, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2005, 2, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2005, 3, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2005, 4, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2005, 5, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2005, 6, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2005, 7, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2005, 8, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2005, 9, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2005, 10, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2005, 11, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2005, 12, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2006, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2006, 

In [26]:
#flatten the list to single column, i tried without flattening that does not make sense
leafn = np.array(leafn).flatten()
leafcn = np.array(leafcn).flatten()

leafn

array([4.19828363, 3.31777078, 4.88835183, ..., 2.71169825, 1.83346201,
       3.52387065])

In [12]:
#there should be no missing values becuase machine does not learn anything from
#that and it also does not how to interpret the data
np.unique(leafn)
np.isnan(leafcn).sum()


0

#### The K nearest method imputer makes more sense to me because i can use four nearby grid cells values
#### to impute the nan values

In [13]:
from sklearn.impute import KNNImputer

# Create a KNN imputer with k=4 (number of neighbors), not required for leafn as it no na values
knn_imputer = KNNImputer(n_neighbors=4)
leafcn= knn_imputer.fit_transform(np.array(leafcn).reshape(-1,1))


#### Data is cleaned, and it can be now fit using machine learning. But, this data should be shuffled
#### otherwise there is specific grid and time and model won't perform better. So, do random arrnagement of the 
#### values keeping the relationships of input and output

In [15]:
import numpy as np

# Assuming leafn and leafcn have the same length
num_samples = len(leafn)
indices = np.arange(num_samples)
np.random.shuffle(indices)

# Shuffle leafn and leafcn using the shuffled indices
shuffled_leafn = leafn[indices]
shuffled_leafcn = leafcn[indices]


#### Data preprocessing and selecting only 1000 rows

In [21]:
# split the dataset into xtrain and ytrain 
from sklearn.model_selection import train_test_split

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(shuffled_leafn, 
                                                    shuffled_leafcn, 
                                                    test_size=0.2, 
                                                    random_state=42)

# #---------use standard scaler to transform the dataset
# from sklearn.preprocessing import StandardScaler

# # Create StandardScaler objects
# train_x_scale= StandardScaler()

# # Fit the scalers to the data and transform the data
X_train= np.array(X_train).reshape(-1, 1)
X_test =  np.array(X_test).reshape(-1, 1)

#for working with ml subsetting only 1000 rows
y_train = X_train[:1000]
y_train = y_train[:1000]


#### This is based model with no hyperparameter changing, no cross validation


In [23]:

#two packages that can be used to instantiate the gaussian model
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

#instantiate the model and tune nothing in the beginning
kernel = ConstantKernel(constant_value = 3, constant_value_bounds=(1e-2, 1e2)) \
              * RBF(length_scale=1, length_scale_bounds=(1e-8, 1e8))
gp_model = GaussianProcessRegressor(kernel=kernel, random_state=42)

# Fit the model to the training data
gp_model.fit(X_train, y_train)

# Evaluate the model on the testing data
score = gp_model.score(X_test, y_test)
print("Model Score:", score)

Model Score: -42.467727788611484


/glade/u/home/sbhattarai/.conda/envs/meds-py/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:434: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/glade/u/home/sbhattarai/.conda/envs/meds-py/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:424: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


#### This is a model with 4 folds cross validation, neaning training 4 times from subsets
#### for hyperparameter tuning, need to consult with Daniel

In [24]:
#use kfold validation to train the model
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error
import time

# Define the number of folds for cross-validation
n_splits = 5

#starttime for model run
start_time = time.time()

# Define the Gaussian Process model
kernel = RBF()
# kernel = ConstantKernel(constant_value = 3, constant_value_bounds=(1e-5, 1e5)) \
#               * RBF(length_scale=1, length_scale_bounds=(1e-5, 1e5))
gp_model_cv = GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer= 4)

# Perform k-fold cross-validation and calculate MAE for each fold
mae_scores = cross_val_score(gp_model_cv, X_train, y_train, cv = n_splits, scoring='neg_mean_absolute_error')

print(mae_scores)
#endtime for model run
end_time = time.time()


[-26.92601934 -27.0464382  -28.11737859 -26.43420394 -25.75277816]


In [25]:
# I don't understand why model only predicting zeros, tried all possible approaches
gp_model_cv.predict(X_test)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.